In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [3]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [4]:
indep=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dep=dataset['Purchased']

In [5]:
#split into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(indep, dep, test_size=1/3, random_state=0)

In [6]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'criterion':['gini', 'entropy'],
            'max_features':['sqrt', 'log2'],
            'n_estimators':[10,100]}
grid=GridSearchCV(RandomForestClassifier(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [8]:
#print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
#print(re)
grid_predictions=grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)
print('The confusion matrix:\n',cm)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)
print('The classification report:\n',clf_report)
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='macro')
print('f1_macro value for best parameter {}:'.format(grid.best_params_),f1_macro)
f1_weighted=f1_score(y_test,grid_predictions,average='weighted')
print('f1_weighted value for best parameter {}:'.format(grid.best_params_),f1_weighted)

The confusion matrix:
 [[79  6]
 [ 4 45]]
The classification report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94        85
           1       0.88      0.92      0.90        49

    accuracy                           0.93       134
   macro avg       0.92      0.92      0.92       134
weighted avg       0.93      0.93      0.93       134

f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 100}: 0.9202380952380953
f1_weighted value for best parameter {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 100}: 0.9256751954513149


In [9]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.9627851140456183)

In [10]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.049653,0.001441,0.011196,0.001347,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.867478,0.889022,0.833323,0.870362,0.923510,0.876739,0.029495,7
1,0.441035,0.119541,0.035614,0.020964,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.867478,0.888107,0.851527,0.925272,0.943699,0.895217,0.034591,3
2,0.036071,0.001525,0.008200,0.000403,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.828959,0.851527,0.906166,0.962264,0.879211,0.048876,6
3,0.578379,0.075714,0.033748,0.016319,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.907177,0.851527,0.944161,0.943699,0.898741,0.042537,2
4,0.054364,0.015334,0.009860,0.002664,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.847141,0.849057,0.851527,0.925272,0.943041,0.883208,0.042000,5
5,0.354521,0.012792,0.025010,0.004652,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.826263,0.888107,0.851527,0.925272,0.943041,0.886842,0.043712,4
6,0.036045,0.000958,0.008807,0.000646,entropy,log2,10,"{'criterion': 'entropy', 'max_features': 'log2...",0.887346,0.849057,0.813179,0.925272,0.905069,0.875984,0.040184,8
7,0.344285,0.011662,0.020061,0.002471,entropy,log2,100,"{'criterion': 'entropy', 'max_features': 'log2...",0.867478,0.888107,0.851527,0.925272,0.962264,0.898930,0.040146,1
